In [1]:
from text_denoiser import TextDenoiser
import torch

c:\Users\willd\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
denoiser = TextDenoiser()
denoiser.load_state_dict(torch.load("../logs/train_test/models/saved_model.pt", map_location=device))

<All keys matched successfully>

In [17]:
texts = ["shovel", "bucket"]
indices = torch.LongTensor(denoiser.vocab(texts)).to(device)

denoiser.embedder(indices)
# Cosine similarity between "shovel" and "bucket" embeddings
print(torch.nn.functional.cosine_similarity(denoiser.embedder(indices[0:1]), denoiser.embedder(indices[1:2])))

# Cosine similarity between "shovel" and "king" embeddings
texts = ["window", "king"]
indices = torch.LongTensor(denoiser.vocab(texts)).to(device)
print(torch.nn.functional.cosine_similarity(denoiser.embedder(indices[0:1]), denoiser.embedder(indices[1:2])))


tensor([-0.0084], grad_fn=<SumBackward1>)
tensor([0.0070], grad_fn=<SumBackward1>)


In [19]:
denoiser.sample(device, 1, 10)

['s <unk> plan sketches spikes managed tiger 1800 their acquire']

In [36]:
from transformers import BertConfig, BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [89]:
# Setup bert pipeline

def get_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs[0]


# text = "shovel bucket"
# tokens = tokenizer.tokenize(text)
# tokens = ["[CLS]"] + tokens + ["[SEP]"]

# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# token_ids = torch.LongTensor(token_ids).unsqueeze(0)

# with torch.no_grad():
#     outputs = model(token_ids)
#     last_hidden_states = outputs[0]

# print(last_hidden_states[0, 1, 0:3])

emb1 = get_embedding("Room", model, tokenizer)
emb2 = get_embedding("bathroom", model, tokenizer)

print(emb1[0, 1].norm())
print(emb1.shape[-1] ** 0.5)

# Cosine similarity between "shovel" and "bucket" embeddings
# print(torch.nn.functional.cosine_similarity(last_hidden_states[0, 1:2], last_hidden_states[0, 2:3]))
# print(model.get_output_embeddings())

# Cosine similarity between embeddings
# print(torch.nn.functional.cosine_similarity(emb1[0, 0:1], emb2[0, 0]))

# inverse process: get text from embedding



tensor(14.1941, grad_fn=<NormBackward1>)
27.712812921102035
